In [1]:
### Find Missing Plants

In [2]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia923, models_eia923
from pudl.eia923 import get_eia923_page
from pudl.eia923 import get_eia923_xlsx, get_eia923_column_map
from pudl.eia923 import get_eia923_plant_info, get_eia923_operator_info
from pudl.pudl import *
from pudl.constants import tab_map_eia923
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [3]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.eia923
%aimport pudl.constants
%aimport pudl.settings
%aimport pudl.models
%aimport pudl.models_eia923

In [4]:
eia923_years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [5]:
eia923_xlsx = get_eia923_xlsx(eia923_years)

Reading EIA 923 spreadsheet data for 2009.
Reading EIA 923 spreadsheet data for 2010.
Reading EIA 923 spreadsheet data for 2011.
Reading EIA 923 spreadsheet data for 2012.
Reading EIA 923 spreadsheet data for 2013.
Reading EIA 923 spreadsheet data for 2014.
Reading EIA 923 spreadsheet data for 2015.
Reading EIA 923 spreadsheet data for 2016.


In [6]:
eia923_plants = get_eia923_operator_info(eia923_years, eia923_xlsx)
eia923_operator_ids = eia923_plants[['operator_id']]

Converting EIA 923 generation_fuel to DataFrame...
Converting EIA 923 boiler_fuel to DataFrame...
Converting EIA 923 generator to DataFrame...


In [7]:
eia923_plants = eia923_plants.dropna(subset=['operator_id'])
eia923_plants.head(2)

,operator_id,operator_name
0,195,Alabama Power Co
1,195,Alabama Power Co


In [8]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'utilities_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc1': int,
                                          'respondent_name_ferc1': str,
                                          'plant_name_ferc1': str,
                                          'plant_id_eia923': int,
                                          'plant_name_eia923': str,
                                          'operator_name_eia923': str,
                                          'operator_id_eia923': int})
output_operator_ids = output_tab_plants[['operator_id_eia923']]
output_operator_ids = output_operator_ids.dropna()

In [9]:
output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia923': 'operator_id'})

In [10]:
merged = eia923_operator_ids.merge(output_operator_ids, indicator=True, how='outer')
merged = merged[merged['_merge'] == 'right_only']
merged = merged.drop_duplicates()
del merged['_merge']

In [11]:
merged.shape

(212, 1)

In [12]:
missing_operators = merged.rename(columns={0: 'operator_id',})
missing_operators.drop_duplicates()
eia923_plants.drop_duplicates('operator_id')
missing_operators["operator_id"] = missing_operators.operator_id.astype(int)
eia923_plants["operator_id"] = eia923_plants.operator_id.astype(int)

In [13]:
missing_operators = pd.merge(missing_operators, eia923_plants, on=['operator_id'], how='left')

In [14]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc1': int,
                                          'respondent_name_ferc1': str,
                                          'plant_name_ferc1': str,
                                          'plant_id_eia923': int,
                                          'plant_name_eia923': str,
                                          'operator_name_eia923': str,
                                          'operator_id_eia923': int})
output_operator_ids = output_tab_plants[['operator_id_eia923', 'operator_name_eia923']]
output_operator_ids = output_operator_ids.dropna()

In [15]:
output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia923': 'operator_id'})

In [16]:
missing_operators = pd.merge(missing_operators, output_operator_ids, on=['operator_id'], how='left')

In [17]:
missing_operators = missing_operators.drop_duplicates('operator_id')
del missing_operators['operator_name']
missing_operators = missing_operators.rename(columns={'operator_name_eia923': 'operator_name'})

In [18]:
missing_operators.head(20)

,operator_id,operator_name
0,99999,State-Fuel Level Increment
1,65,Abitibi Consolidated-Lufkin
2,163,AERA Energy LLC-Oxford
3,178,AES Placerita Inc
4,179,Agrium US Inc
5,358,Amalgamated Sugar Co LLC
6,402,Altamont Power LLC
7,404,Alsip Paper Condominium Assn
8,431,Alyeska Seafoods Inc
9,877,Art Institute of Chicago


In [19]:
missing_operators.to_csv('../id_mapping/missing_operators.csv', index=False)